In [2]:
import csv
import cv2
import numpy as np
import os
import glob
from sklearn.utils import shuffle
from collections import OrderedDict
from shutil import copyfile

from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.preprocessing import image
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.optimizers import SGD
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

Using TensorFlow backend.


In [ ]:
#ensure GPU
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
#check dim ordering
print(k.image_data_format())

In [ ]:
# a bunch of junk code for pushing around earlier data that I'm not sure I won't need again...

# find . -type f -name '*.png' -delete

# !! ls data/TrafficLightDataset/working_data/

# with open('./data/TrafficLightDataset/labels_num.csv') as labels:
#     c_labels = OrderedDict(csv.reader(labels))

# X_labels = np.array(list(c_labels.keys()))
# Y = np.array(list(c_labels.values()))
# shuffle(X_labels, Y)

# images = []
# for label in X_labels:
#     loc = "/home/dieslow/WORKSPACE/PROJECTS/CARZ/nanodegree/Capstone/ros/src/tl_detector/data/TrafficLightDataset/image_data/" + label + ".png"
#     image = cv2.imread(loc, 1)
# #     image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB) 
#     # May not need to do resizing... BUT SHOULD VIEW IMAGES
# #     resized_image = cv2.resize(image, (256, 256)) 
#     images.append(image)
# X = np.array(images)
# print(X.shape)

# train_val_index = int(np.floor(len(Y) * 0.85))

# D = list(zip(X, Y))
# D_train = D[:train_val_index]
# D_val = D[train_val_index:]
# X_train, Y_train = zip(*D_train)
# print(len(X_train))
# X_val, Y_val = zip(*D_val)
# print(len(X_val))

# images = []
# for x, y in D_train:
#     loc = "/home/dieslow/WORKSPACE/PROJECTS/CARZ/nanodegree/" + \
#           "Capstone/ros/src/tl_detector/data/TrafficLightDataset/working_data/train/" + \
#           str(y) + "/" + str(np.random.randint(100000000, 999999999)) + ".png"
#     cv2.imwrite(loc, x)
    
# for x, y in D_val:
#     loc = "/home/dieslow/WORKSPACE/PROJECTS/CARZ/nanodegree/" + \
#           "Capstone/ros/src/tl_detector/data/TrafficLightDataset/working_data/val/" + \
#           str(y) + "/" + str(np.random.randint(100000000, 999999999)) + ".png"
#     cv2.imwrite(loc, x)

In [3]:
# helper stolen 
def get_nb_files(directory):
  """Get number of files by searching directory recursively"""
  if not os.path.exists(directory):
    return 0
  cnt = 0
  for r, dirs, files in os.walk(directory):
    for dr in dirs:
      cnt += len(glob.glob(os.path.join(r, dr + "/*")))
  return cnt

In [4]:
IMG_HEIGHT, IMG_WIDTH = 299, 299

# TRAIN_DATA_DIR = "data/TrafficLightDataset/working_data/train"
# VAL_DATA_DIR = "data/TrafficLightDataset/working_data/val"
TRAIN_DATA_DIR = "./data/TrafficLightDataset/big_working_data/train"
VAL_DATA_DIR = "./data/TrafficLightDataset/big_working_data/val"
TEST_DATA_DIR = "./data/TrafficLightDataset/big_working_data/test"

NB_TRAIN_SAMPLES = get_nb_files(TRAIN_DATA_DIR)
NB_VAL_SAMPLES = get_nb_files(VAL_DATA_DIR)
NB_TEST_SAMPLES = get_nb_files(TEST_DATA_DIR)

BATCH_SIZE = 40
EPOCHS_TRANSFER = 5
EPOCHS_FINE_TUNE = 5

TRAIN_STEPS_PER_EPOCH = int(np.ceil(NB_TRAIN_SAMPLES / BATCH_SIZE))
VAL_STEPS_PER_EPOCH = int(np.ceil(NB_VAL_SAMPLES / BATCH_SIZE))
TEST_STEPS_PER_EPOCH = int(np.ceil(NB_TEST_SAMPLES / BATCH_SIZE))

FROZEN_LAYERS = 172

In [5]:
# Initiate the train and test generators with data Augumentation 
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
#     rescale = 1./255,
#     horizontal_flip = True,
#     zoom_range = 0.4,
#     width_shift_range = 0.3,
#     height_shift_range=0.3,
#     rotation_range=20
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
#     horizontal_flip = True,
#     rescale = 1./255,
#     fill_mode = "nearest"
#     zoom_range = 0.3,
#     width_shift_range = 0.3,
#     height_shift_range=0.3,
#     rotation_range=30
)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DATA_DIR,
    target_size = (IMG_HEIGHT, IMG_WIDTH),
    batch_size = BATCH_SIZE)

validation_generator = test_datagen.flow_from_directory(
    VAL_DATA_DIR,
    batch_size = BATCH_SIZE,
    target_size = (IMG_HEIGHT, IMG_WIDTH))

test_generator = test_datagen.flow_from_directory(
    VAL_DATA_DIR,
    batch_size = BATCH_SIZE,
    target_size = (IMG_HEIGHT, IMG_WIDTH))


Found 12319 images belonging to 4 classes.
Found 2728 images belonging to 4 classes.
Found 2728 images belonging to 4 classes.


## Notes
-rescale and fill dont seem to impact much
-neither does class mode or class weight

In [50]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape = (IMG_HEIGHT, IMG_WIDTH, 3))

# + global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# + FC layer
x = Dense(1024, activation='relu')(x)

# + log laye
predictions = Dense(4, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# freeze all convolutional layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (most places, including the inceptv3 paper recommended RMSProp, 
#  but I had a lot of premature convergence issues with it. to be fair, I switched to
#  adam experimenting around before I spent anytime optimizing LR, so who knows. converges
#  quick enough for Anand's data as it is)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Save the model according to the conditions  
checkpoint = ModelCheckpoint("inceptionv3.h5", monitor='val_loss', verbose=1, save_best_only=True, 
                             save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')

# training for transfer learning
model.fit_generator(
    train_generator,
    steps_per_epoch = TRAIN_STEPS_PER_EPOCH,
    epochs = EPOCHS_TRANSFER,
    validation_data = validation_generator,
    validation_steps = VAL_STEPS_PER_EPOCH,
    callbacks = [early, checkpoint],
    verbose=True, 
    class_weight='auto',
    use_multiprocessing=True)

# start fine-tuning conv layers in addition to FC. 
# freeze the bottom FROZEN_LAYERS layers, train the rest
for layer in model.layers[:FROZEN_LAYERS]:
   layer.trainable = False
for layer in model.layers[FROZEN_LAYERS:]:
   layer.trainable = True

# compile
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

# training for fine-tuning conv layers
model.fit_generator(
    train_generator,
    steps_per_epoch = TRAIN_STEPS_PER_EPOCH,
    epochs = EPOCHS_FINE_TUNE,
    validation_data = validation_generator,
    validation_steps = VAL_STEPS_PER_EPOCH,
    callbacks = [early, checkpoint], #
    verbose=True,
    class_weight='auto',
    use_multiprocessing=True)


print(model.metrics_names)
model.evaluate_generator(
    test_generator,
    steps = TEST_STEPS_PER_EPOCH,
    use_multiprocessing=True)

Found 12319 images belonging to 4 classes.
Found 2728 images belonging to 4 classes.
Found 2614 images belonging to 4 classes.
Epoch 1/5
308/308 [==============================] - 262s - loss: 0.3946 - acc: 0.8688 - val_loss: 0.2313 - val_acc: 0.9183
Epoch 2/5
308/308 [==============================] - 204s - loss: 0.2179 - acc: 0.9243 - val_loss: 0.2308 - val_acc: 0.9274
Epoch 3/5
308/308 [==============================] - 222s - loss: 0.1785 - acc: 0.9364 - val_loss: 0.1098 - val_acc: 0.9666
Epoch 4/5
308/308 [==============================] - 229s - loss: 0.1546 - acc: 0.9450 - val_loss: 0.1134 - val_acc: 0.9644
Epoch 5/5
308/308 [==============================] - 232s - loss: 0.1518 - acc: 0.9426 - val_loss: 0.1026 - val_acc: 0.9637
Epoch 1/5
308/308 [==============================] - 253s - loss: 0.0684 - acc: 0.9785 - val_loss: 0.0436 - val_acc: 0.9835
Epoch 2/5
308/308 [==============================] - 251s - loss: 0.0291 - acc: 0.9918 - val_loss: 0.0302 - val_acc: 0.9905
Epoch

[1.0358809883027198, 0.77505740999901851]

In [ ]:
model.count_params()